# VacationPy
----



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests
import json
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = "../output_data/weather_data.csv"
weather_data = pd.read_csv(weather_data)
weather_data

,Unnamed: 0,City,Country,Date,Lat,Lon,Max Temp,Cloudiness,Wind Speed,Humidity
0,0,port alfred,ZA,1586652405,-33.59,26.89,18.33,40,1.34,83
1,1,hilo,US,1586652298,19.73,-155.09,23.78,90,4.10,69
2,2,hermanus,ZA,1586652266,-34.42,19.23,16.97,100,8.05,93
3,3,puerto ayora,EC,1586652739,-0.74,-90.35,27.16,75,5.10,74
4,4,ndele,CF,1586652739,8.41,20.65,23.58,99,1.57,73
...,...,...,...,...,...,...,...,...,...,...
531,531,salto del guaira,PY,1586653059,-24.06,-54.31,16.95,0,1.03,49
532,532,khandyga,RU,1586653060,62.67,135.60,-0.77,100,2.80,96
533,533,bradford,GB,1586652896,53.79,-1.75,8.50,5,1.50,93
534,534,burns lake,CA,1586653061,54.23,-125.75,5.32,14,1.14,45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Humidity Heat Map
location_data = weather_data[["Lat","Lon"]]
data = weather_data["Humidity"].astype(float)

#Figure Layout
figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)
heatmap = gmaps.heatmap_layer(location_data, weights=data,
                              dissipating=False, max_intensity=86,
                              point_radius=2)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [4]:
# Between degrees celcius: 21C - 27C; Wind speed less than 10mph and 0 cloudiness
vacation_df = weather_data.loc[(weather_data["Max Temp"] < 27) 
                                & (weather_data["Max Temp"] > 21) 
                                & (weather_data["Wind Speed"] < 10)
                                & (weather_data["Cloudiness"] == 0)]
                                
vacation_df.reset_index(inplace=True)
del vacation_df["index"]
del vacation_df["Unnamed: 0"]
vacation_df



,City,Country,Date,Lat,Lon,Max Temp,Cloudiness,Wind Speed,Humidity
0,adrar,MR,1586652501,20.50,-10.07,26.93,0,4.88,17
1,nouakchott,MR,1586652893,18.09,-15.98,23.84,0,7.75,53
2,ballina,AU,1586652897,-28.87,153.57,21.99,0,8.20,49
3,narasannapeta,IN,1586652970,18.42,84.05,24.95,0,2.14,88
4,rambha,IN,1586652975,19.52,85.10,25.02,0,1.42,79
5,gold coast,AU,1586652978,-28.00,153.43,24.52,0,7.70,44
6,biswan,IN,1586652986,27.50,81.00,24.50,0,1.66,25
7,godda,IN,1586652995,24.83,87.22,23.96,0,0.97,35
8,tura,IN,1586652426,25.52,90.22,21.82,0,2.17,46
9,cockburn town,TC,1586653027,21.46,-71.14,26.45,0,6.14,77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:

# base url, params & containers
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "hotel"
target_radius = 5000
hotel_df = []
hotel_name = []
address = []

#loop through all the cities from the imported dataset .loc[i,4] .loc[i,5]
for  i in range(len(vacation_df)):
    
    lat = vacation_df.loc[i]["Lat"]
    lon = vacation_df.loc[i]["Lon"]

    # set up a parameters dictionary
    params = {
        "location": f'{lat},{lon}',
        "keyword": target_search,
        "radius": target_radius,
        #"type": target_type,
        "key": g_key}


    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    #pprint(response)
    
    try:
        #append to lists
        hotel = response['results'][0]['name']
        vicinity = response['results'][0]['vicinity']
        #hotel_df.append([hotel_name, address])
        print(hotel,vicinity)
    
        hotel_name.append(hotel)
        address.append(vicinity)
        
        
    except Exception:
        #print city name that was not found
        hotel_name.append('NaN')
        address.append('NaN')
        print(f"{i} hotel not found!")


0 hotel not found!
AZALAÏ HOTEL NOUAKCHOTT (MARHABA ) Avenue Gamal Abdel Nasser، Nouakchott
Ballina Travellers Lodge 36 Tamar St, Ballina
Sri laxmi lodge Main Road, Maruthi Nagar, Narasannapeta
OTDC Panthanivas Rambha
Hilton Surfers Paradise Hotel & Residences Gold Coast, 6 Orchid Ave, Surfers Paradise
Gausiya Hotel Biswan
HOTEL TUSH TAYA RESIDENCY NEAR MUFISIL THANA, Jamua
Hotel Polo Orchid Araimile, Tura
Bohio Dive Resort Cockburn Town


In [6]:
hotel_name

['NaN',
 'AZALAÏ HOTEL NOUAKCHOTT (MARHABA )',
 'Ballina Travellers Lodge',
 'Sri laxmi lodge',
 'OTDC Panthanivas',
 'Hilton Surfers Paradise Hotel & Residences',
 'Gausiya Hotel',
 'HOTEL TUSH TAYA RESIDENCY',
 'Hotel Polo Orchid',
 'Bohio Dive Resort']

In [7]:
vacation_df["Hotel Name"] = hotel_name
vacation_df["Address"] = address

/Users/kateengard/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/kateengard/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
hotel_df = vacation_df[vacation_df["Hotel Name"] != "NaN"]
hotel_df

,City,Country,Date,Lat,Lon,Max Temp,Cloudiness,Wind Speed,Humidity,Hotel Name,Address
1,nouakchott,MR,1586652893,18.09,-15.98,23.84,0,7.75,53,AZALAÏ HOTEL NOUAKCHOTT (MARHABA ),Avenue Gamal Abdel Nasser، Nouakchott
2,ballina,AU,1586652897,-28.87,153.57,21.99,0,8.20,49,Ballina Travellers Lodge,"36 Tamar St, Ballina"
3,narasannapeta,IN,1586652970,18.42,84.05,24.95,0,2.14,88,Sri laxmi lodge,"Main Road, Maruthi Nagar, Narasannapeta"
4,rambha,IN,1586652975,19.52,85.10,25.02,0,1.42,79,OTDC Panthanivas,Rambha
5,gold coast,AU,1586652978,-28.00,153.43,24.52,0,7.70,44,Hilton Surfers Paradise Hotel & Residences,"Gold Coast, 6 Orchid Ave, Surfers Paradise"
6,biswan,IN,1586652986,27.50,81.00,24.50,0,1.66,25,Gausiya Hotel,Biswan
7,godda,IN,1586652995,24.83,87.22,23.96,0,0.97,35,HOTEL TUSH TAYA RESIDENCY,"NEAR MUFISIL THANA, Jamua"
8,tura,IN,1586652426,25.52,90.22,21.82,0,2.17,46,Hotel Polo Orchid,"Araimile, Tura"
9,cockburn town,TC,1586653027,21.46,-71.14,26.45,0,6.14,77,Bohio Dive Resort,Cockburn Town


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…